# 05 abr 2018

O objetivo de hoje é gerar e popular o banco de dados

In [1]:
import pandas as pd
import numpy as np
import mysql.connector as mysql

In [2]:
df = pd.read_csv(
    '../data/2018-04-04-candidaturas-tse.xz',
    low_memory=False
)

Conexão com o banco de dados local

In [3]:
config = {
    'user': 'root',
    'password': '',
    'host': '127.0.0.1',
    'database': 'laddres'
}

cnx = mysql.connect(**config)
cnx.close()

## Estados

Popular a tabela com os estados

In [126]:
estados = pd.DataFrame({
    'sigla': [
        'AC',
        'AL',
        'AM',
        'AP',
        'BA',
        'CE',
        'DF',
        'ES',
        'GO',
        'MA',
        'MG',
        'MS',
        'MT',
        'PA',
        'PB',
        'PE',
        'PI',
        'PR',
        'RJ',
        'RN',
        'RO',
        'RR',
        'RS',
        'SC',
        'SE',
        'SP',
        'TO'
    ],
    'nome': [
        'Acre',
        'Alagoas',
        'Amazonas',
        'Amapá',
        'Bahia',
        'Ceará',
        'Distrito Federal',
        'Espírito Santos',
        'Goiás',
        'Maranhão',
        'Minas Gerais',
        'Mato Grosso do Sul',
        'Mato Grosso',
        'Pará',
        'Paraíba',
        'Pernambuco',
        'Piauí',
        'Paraná',
        'Rio de Janeiro',
        'Rio Grande do Norte',
        'Rondônia',
        'Roraima',
        'Rio Grande do Sul',
        'Santa Catarina',
        'Sergipe',
        'São Paulo',
        'Tocantins'
    ]
})

In [127]:
cnx = mysql.connect(**config)
cursor = cnx.cursor()

for estado in estados.itertuples():
    novo_estado = ("INSERT INTO estado "
                   "(sigla, nome) "
                   "VALUES (%(sigla)s, %(nome)s)")
    cursor.execute(novo_estado, {
        'sigla': estado.sigla,
        'nome': estado.nome
    })
    cnx.commit()

cursor.close()
cnx.close()

## Cidade

In [128]:
cidades = (
    df[[
        'SIGLA_UE',
        'DESCRICAO_UE',
        'SIGLA_UF'
    ]]
    .drop_duplicates('SIGLA_UE', keep='last')
    .sort_values(by=['SIGLA_UE'], ascending=True, na_position='first')
    .rename(columns = {
        'SIGLA_UE': 'codigo_tse',
        'DESCRICAO_UE': 'nome',
        'SIGLA_UF': 'estado'
    })
)[:-28]

In [129]:
cnx = mysql.connect(**config)
cursor = cnx.cursor()

for cidade in cidades.itertuples():
    # Buscar id do estado
    query = ("SELECT id FROM estado WHERE sigla = '" + cidade.estado + "'")
    cursor.execute(query)
    estado_id = cursor.fetchone()[0]
    # Cadastrar cidade
    nova_cidade = ("INSERT INTO cidade "
                   "(codigo_tse, nome, estado_id) "
                   "VALUES (%(codigo_tse)s, %(nome)s, %(estado_id)s)")
    cursor.execute(nova_cidade, {
        'codigo_tse': cidade.codigo_tse,
        'nome': cidade.nome,
        'estado_id': estado_id
    })
    cnx.commit()

cursor.close()
cnx.close()

## Nacionalidade

In [117]:
nacionalidades = (
    df[[
        'CODIGO_NACIONALIDADE',
        'DESCRICAO_NACIONALIDADE'
    ]]
    .drop_duplicates('CODIGO_NACIONALIDADE', keep='last')
    .sort_values(by=['CODIGO_NACIONALIDADE'], ascending=True, na_position='first')
    .rename(columns = {
        'CODIGO_NACIONALIDADE': 'codigo_tse',
        'DESCRICAO_NACIONALIDADE': 'nome'
    })
)[:-1]

In [118]:
nacionalidades

,codigo_tse,nome
401701,0,NÃO INFORMADA
1837258,1,BRASILEIRA NATA
1836773,2,BRASILEIRA (NATURALIZADA)
1830351,3,PORTUGUESA COM IGUALDADE DE DIREITOS
1836916,4,ESTRANGEIRO


In [130]:
cnx = mysql.connect(**config)
cursor = cnx.cursor()

for nacionalidade in nacionalidades.itertuples():
    novo_nacionalidade = ("INSERT INTO nacionalidade "
                   "(id, nome) "
                   "VALUES (%(codigo_tse)s, %(nome)s)")
    cursor.execute(novo_nacionalidade, {
        'codigo_tse': nacionalidade.codigo_tse,
        'nome': nacionalidade.nome
    })
    cnx.commit()

cursor.close()
cnx.close()

# Grau de Instrução

In [121]:
graus_instrucao = (
    df[[
        'COD_GRAU_INSTRUCAO',
        'DESCRICAO_GRAU_INSTRUCAO'
    ]]
    .drop_duplicates('COD_GRAU_INSTRUCAO', keep='last')
    .sort_values(by=['COD_GRAU_INSTRUCAO'], ascending=True, na_position='first')
    .rename(columns = {
        'COD_GRAU_INSTRUCAO': 'codigo_tse',
        'DESCRICAO_GRAU_INSTRUCAO': 'descricao'
    })
)[:-2]

In [122]:
graus_instrucao

,codigo_tse,descricao
410000,0,NÃO INFORMADO
1790642,1,ANALFABETO
1837248,2,LÊ E ESCREVE
1837257,3,ENSINO FUNDAMENTAL INCOMPLETO
1837254,4,ENSINO FUNDAMENTAL COMPLETO
1837243,5,ENSINO MÉDIO INCOMPLETO
1837258,6,ENSINO MÉDIO COMPLETO
1837250,7,SUPERIOR INCOMPLETO
1837255,8,SUPERIOR COMPLETO


In [131]:
cnx = mysql.connect(**config)
cursor = cnx.cursor()

for grau in graus_instrucao.itertuples():
    novo_grau = ("INSERT INTO grau_instrucao "
                   "(id, descricao) "
                   "VALUES (%(codigo_tse)s, %(descricao)s)")
    cursor.execute(novo_grau, {
        'codigo_tse': grau.codigo_tse,
        'descricao': grau.descricao
    })
    cnx.commit()

cursor.close()
cnx.close()

## Ocupação

In [133]:
ocupacoes = (
    df[[
        'CODIGO_OCUPACAO',
        'DESCRICAO_OCUPACAO'
    ]]
    .drop_duplicates('CODIGO_OCUPACAO', keep='last')
    .sort_values(by=['CODIGO_OCUPACAO'], ascending=True, na_position='first')
    .rename(columns = {
        'CODIGO_OCUPACAO': 'codigo_tse',
        'DESCRICAO_OCUPACAO': 'descricao'
    })
)[:-2]

In [134]:
ocupacoes

,codigo_tse,descricao
401787,0,NÃO INFORMADA
1836784,101,ENGENHEIRO
1834397,102,ARQUITETO
1836343,103,AGRÔNOMO
1829197,104,QUÍMICO
1792908,106,FÍSICO
1807883,107,GEÓLOGO
1837224,109,AGENTE DE SAÚDE E SANITARISTA
1834161,110,AGENTE POSTAL
1837247,111,MÉDICO


In [135]:
cnx = mysql.connect(**config)
cursor = cnx.cursor()

for ocupacao in ocupacoes.itertuples():
    nova_ocupacao = ("INSERT INTO ocupacao "
                   "(id, descricao) "
                   "VALUES (%(codigo_tse)s, %(descricao)s)")
    cursor.execute(nova_ocupacao, {
        'codigo_tse': ocupacao.codigo_tse,
        'descricao': ocupacao.descricao
    })
    cnx.commit()

cursor.close()
cnx.close()

## Eleição

In [136]:
eleicoes = (
    df[[
        'ANO_ELEICAO',
        'DESCRICAO_ELEICAO'
    ]]
    .drop_duplicates('DESCRICAO_ELEICAO', keep='last')
    .sort_values(by=['ANO_ELEICAO'], ascending=True, na_position='first')
    .rename(columns = {
        'ANO_ELEICAO': 'ano',
        'DESCRICAO_ELEICAO': 'descricao'
    })
)

In [137]:
eleicoes

,ano,descricao
401788,2004,ELEICOES 2004
422578,2006,ELEICOES 2006
578196,2008,ELEIÇÕES SUPLEMENTARES
800173,2008,PLEBISCITO
807094,2008,ELEIÇÕES 2008
807109,2008,ELEIÇÕES SUPLEMENTARES 2008
829686,2010,ELEIÇÕES 2010
1200422,2012,MAJORITÁRIA - PEDRAS ALTAS
1200426,2012,MAJORITÁRIA - TAQUARI
1200434,2012,MAJORITÁRIA - EUGÊNIO DE CASTRO


In [139]:
cnx = mysql.connect(**config)
cursor = cnx.cursor()

for eleicao in eleicoes.itertuples():
    nova_eleicao = ("INSERT INTO eleicao "
                   "(ano, descricao) "
                   "VALUES (%(ano)s, %(descricao)s)")
    cursor.execute(nova_eleicao, {
        'ano': eleicao.ano,
        'descricao': eleicao.descricao
    })
    cnx.commit()

cursor.close()
cnx.close()

## Cargos

In [252]:
cargos = (
    df[[
        'CODIGO_CARGO',
        'DESCRICAO_CARGO'
    ]]
    .drop_duplicates('CODIGO_CARGO', keep='last')
    .sort_values(by=['CODIGO_CARGO'], ascending=True, na_position='first')
    .rename(columns = {
        'CODIGO_CARGO': 'codigo_tse',
        'DESCRICAO_CARGO': 'nome'
    })
)[:-1]

In [253]:
cargos

,codigo_tse,nome
1317068,1,PRESIDENTE
1317066,2,VICE-PRESIDENTE
1339715,3,GOVERNADOR
1339703,4,VICE-GOVERNADOR
1339717,5,SENADOR
1339756,6,DEPUTADO FEDERAL
1339755,7,DEPUTADO ESTADUAL
1319152,8,DEPUTADO DISTRITAL
1339685,9,1º SUPLENTE
1339752,10,2º SUPLENTE


In [145]:
cnx = mysql.connect(**config)
cursor = cnx.cursor()

for cargo in cargos.itertuples():
    novo_cargo = ("INSERT INTO cargo "
                   "(id, nome) "
                   "VALUES (%(codigo_tse)s, %(nome)s)")
    cursor.execute(novo_cargo, {
        'codigo_tse': cargo.codigo_tse,
        'nome': cargo.nome
    })
    cnx.commit()

cursor.close()
cnx.close()

## Partidos

In [146]:
partidos = (
    df[[
        'NUMERO_PARTIDO',
        'SIGLA_PARTIDO',
        'NOME_PARTIDO'
    ]]
    .drop_duplicates('NUMERO_PARTIDO', keep='last')
    .sort_values(by=['NUMERO_PARTIDO'], ascending=True, na_position='first')
    .rename(columns = {
        'NUMERO_PARTIDO': 'numero',
        'SIGLA_PARTIDO': 'sigla',
        'NOME_PARTIDO': 'nome',
    })
)[:-2]

In [147]:
partidos

,numero,sigla,nome
1837240,10,PRB,PARTIDO REPUBLICANO BRASILEIRO
1837255,11,PP,PARTIDO PROGRESSISTA
1837229,12,PDT,PARTIDO DEMOCRÁTICO TRABALHISTA
1837253,13,PT,PARTIDO DOS TRABALHADORES
1837256,14,PTB,PARTIDO TRABALHISTA BRASILEIRO
1837257,15,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO
1824733,16,PSTU,PARTIDO SOCIALISTA DOS TRABALHADORES UNIFICADO
1837217,17,PSL,PARTIDO SOCIAL LIBERAL
1836945,18,REDE,REDE SUSTENTABILIDADE
1837140,19,PTN,PARTIDO TRABALHISTA NACIONAL


In [148]:
cnx = mysql.connect(**config)
cursor = cnx.cursor()

for partido in partidos.itertuples():
    novo_partido = ("INSERT INTO partido "
                   "(numero, sigla, nome) "
                   "VALUES (%(numero)s, %(sigla)s, %(nome)s)")
    cursor.execute(novo_partido, {
        'numero': partido.numero,
        'sigla': partido.sigla,
        'nome': partido.nome
    })
    cnx.commit()

cursor.close()
cnx.close()

## Situação da Candidatura

In [149]:
situacoes_candidatura = (
    df[[
        'COD_SITUACAO_CANDIDATURA',
        'DES_SITUACAO_CANDIDATURA'
    ]]
    .drop_duplicates('COD_SITUACAO_CANDIDATURA', keep='last')
    .sort_values(by=['COD_SITUACAO_CANDIDATURA'], ascending=True, na_position='first')
    .rename(columns = {
        'COD_SITUACAO_CANDIDATURA': 'codigo_tse',
        'DES_SITUACAO_CANDIDATURA': 'descricao'
    })
)[:-1]

In [150]:
situacoes_candidatura

,codigo_tse,descricao
1826796,10,CASSADO
616879,11,IMPUGNADO
1836321,13,NÃO CONHECIMENTO DO PEDIDO
1837252,14,INDEFERIDO
1837030,16,DEFERIDO COM RECURSO
1805071,17,PENDENTE DE JULGAMENTO
1828962,18,CASSADO COM RECURSO
1684173,19,CANCELADO COM RECURSO
1837258,2,DEFERIDO
1836955,4,INDEFERIDO COM RECURSO


In [152]:
cnx = mysql.connect(**config)
cursor = cnx.cursor()

for situacao in situacoes_candidatura.itertuples():
    nova_situacao = ("INSERT INTO situacao_candidatura "
                   "(id, descricao) "
                   "VALUES (%(codigo_tse)s, %(descricao)s)")
    cursor.execute(nova_situacao, {
        'codigo_tse': situacao.codigo_tse,
        'descricao': situacao.descricao
    })
    cnx.commit()

cursor.close()
cnx.close()

## Resultado de Candidatura

In [156]:
resultados_candidatura = (
    df[[
        'COD_SIT_TOT_TURNO',
        'DESC_SIT_TOT_TURNO'
    ]]
    .drop_duplicates('COD_SIT_TOT_TURNO', keep='last')
    .sort_values(by=['COD_SIT_TOT_TURNO'], ascending=True, na_position='first')
    .rename(columns = {
        'COD_SIT_TOT_TURNO': 'codigo_tse',
        'DESC_SIT_TOT_TURNO': 'descricao'
    })
)[3:15]

In [157]:
resultados_candidatura

,codigo_tse,descricao
1837255,1,ELEITO
826076,10,SUBSTITUÍDO
829630,11,INDEFERIDO COM RECURSO
807433,12,CASSADO COM RECURSO
1837251,2,ELEITO POR QP
1837219,3,ELEITO POR MÉDIA
1837246,4,NÃO ELEITO
1837258,5,SUPLENTE
1829480,6,2º TURNO
825631,7,RENÚNCIA/FALECIMENTO/CASSAÇÃO APÓS A ELEIÇÃO


In [158]:
cnx = mysql.connect(**config)
cursor = cnx.cursor()

for resultado in resultados_candidatura.itertuples():
    novo_resultado = ("INSERT INTO resultado_candidatura "
                   "(id, descricao) "
                   "VALUES (%(codigo_tse)s, %(descricao)s)")
    cursor.execute(novo_resultado, {
        'codigo_tse': resultado.codigo_tse,
        'descricao': resultado.descricao
    })
    cnx.commit()

cursor.close()
cnx.close()

## Candidatos

In [4]:
candidatos = (
    df[[
        'NOME_CANDIDATO',
        'DATA_NASCIMENTO',
        'CPF_CANDIDATO',
        'NUM_TITULO_ELEITORAL_CANDIDATO',
        'NM_EMAIL',
        'NOME_MUNICIPIO_NASCIMENTO',
        'CODIGO_NACIONALIDADE',
        'COD_GRAU_INSTRUCAO',
        'CODIGO_OCUPACAO',
    ]]
    .drop_duplicates('CPF_CANDIDATO', keep='last')
    # .sort_values(by=['NOME_CANDIDATO'], ascending=True, na_position='first')
    .rename(columns = {
        'NOME_CANDIDATO': 'nome',
        'DATA_NASCIMENTO': 'data_nascimento',
        'CPF_CANDIDATO': 'cpf',
        'NUM_TITULO_ELEITORAL_CANDIDATO': 'titulo_eleitoral',
        'NM_EMAIL': 'email',
        'NOME_MUNICIPIO_NASCIMENTO': 'cidade',
        'CODIGO_NACIONALIDADE': 'nacionalidade',
        'COD_GRAU_INSTRUCAO': 'grau_instrucao',
        'CODIGO_OCUPACAO': 'ocupacao'
    })
)

In [5]:
candidatos

,nome,data_nascimento,cpf,titulo_eleitoral,email,cidade,nacionalidade,grau_instrucao,ocupacao
0,JOÃO DE ALMEIDA,26081957,18139124249,000356682496,NaN,TENENTE PORTELA,1,4,601
1,PAULO CÉSAR FERREIRA DE ARAUJO,08041957,07857136204,000525772445,NaN,ACRELÂNDIA,1,3,601
3,ERAIDES CAETANO DE SOUZA,26081957,40917860900,002817512496,NaN,ACRELÂNDIA,1,0,601
12,DAMARIS DE OLIVEIRA,24091966,49488929200,000968162410,NaN,UBIRATÃ,1,6,999
14,EDILENE SILVA DOS REIS,07041971,38950707268,020802772232,NaN,CAARAPO,1,3,999
16,EDNA BERNARDINO E SILVA,29061958,34006451253,001738842445,NaN,GUARACI,1,8,143
18,ERACIDES CAETANO DE SOUZA,14011950,17630207972,002817542437,NaN,TAIO,1,6,999
19,ESTEVÃO DE SOUZA FERREIRA,26121978,61072621215,003157012461,NaN,RIO BRANCO,1,6,999
20,FRANCISCO ANTONIO DA SILVA DIAS,15061964,21580430244,000764552488,NaN,BOCA DO ACRE,1,7,143
23,IVAN DA SILVA FERREIRA,23041974,44378793253,002187552488,NaN,SENA MADUREIRA,1,6,999


### Observações

Será necessário checar algumas coisas para podermos preencher corretamente todas as entradas

**Email**

Checar se o `type` é do tipo `str`, caso contrário, será nulo

exemplo: `type(candidatos.iloc[1000000].email) == str`

In [183]:
print(candidatos.iloc[1000000].email)
print(type(candidatos.iloc[1000000].email) == str)

print(candidatos.iloc[0].email)
print(type(candidatos.iloc[0].email) == str)

VITORSOURE@HOTMAIL.COM
True
nan
False


In [6]:
def email_valido(email):
    return type(email) == str

**Data de Nascimento**

Existem diversos tipos de data de nascimento

```
ANO_ELEICAO     DATA_NASCIMENTO
2004            01061949
2006            08/04/1945
2008            26-OCT-59
2010            03-JAN-34
2012            15/02/1958
2014            12/10/1965
2016            06/08/1969
```

In [7]:
def formatar_data(data):
    meses = {
        'JAN': '01',
        'FEB': '02',
        'MAR': '03',
        'APR': '04',
        'MAY': '05',
        'JUN': '06',
        'JUL': '07',
        'AUG': '08',
        'SEP': '09',
        'OCT': '10',
        'NOV': '11',
        'DEC': '12',
    }
    
    if len(data) == 8:
        return data[4:8] + '-' + data[2:4] + '-' + data[0:2]
    elif '/' in data:
        return data[6:10] + '-' + data[3:5] + '-' + data[0:2]
    elif '-' in data:
        dia = data[0:2]
        mes = meses[data[3:6]]
        ano = ('19' + data[7:9]) if (int(data[7:9]) > 18) else ('20' + data[7:9])
        return ano + '-' + mes + '-' + dia
    else:
        print("formatar_data: " + data)
        return None

print(formatar_data('01061949'))
print(formatar_data('08/04/1945'))
print(formatar_data('26-OCT-59'))
print(formatar_data('26OCT59'))

1949-06-01
1945-04-08
1959-10-26
formatar_data: 26OCT59
None


**Elementos indesejados**

Vamos eliminar as duas entradas problemáticas para evitar transtornos

In [8]:
df_candidatos = df[
    (df['CPF_CANDIDATO'] != '#NULO#') & (df['CODIGO_CARGO'] != 91)
    ].drop(df.index[[653018, 743740]])

In [9]:
df_candidatos

,ANO_ELEICAO,CODIGO_CARGO,CODIGO_COR_RACA,CODIGO_ESTADO_CIVIL,CODIGO_LEGENDA,CODIGO_MUNICIPIO_NASCIMENTO,CODIGO_NACIONALIDADE,CODIGO_OCUPACAO,CODIGO_SEXO,COD_GRAU_INSTRUCAO,...,NUMERO_CANDIDATO,NUMERO_PARTIDO,NUM_TITULO_ELEITORAL_CANDIDATO,NUM_TURNO,SEQUENCIAL_CANDIDATO,SIGLA_LEGENDA,SIGLA_PARTIDO,SIGLA_UE,SIGLA_UF,SIGLA_UF_NASCIMENTO
0,2004,11,NaN,3,5,-1,1,601,2,4,...,13.0,13,000356682496,1,6,#NE#,PT,01120,AC,RS
1,2004,11,NaN,3,1,-1,1,601,2,3,...,14.0,14,000525772445,1,1,#NE#,PTB,01120,AC,AC
2,2004,11,NaN,3,12,-1,1,143,2,8,...,45.0,45,002053882488,1,27,#NE#,PSDB,01120,AC,PR
3,2004,12,NaN,3,7,-1,1,601,2,0,...,13.0,40,002817512496,1,7,#NE#,PSB,01120,AC,AC
4,2004,12,NaN,5,3,-1,1,143,2,8,...,14.0,14,002600402445,1,2,#NE#,PTB,01120,AC,AC
5,2004,12,NaN,3,14,-1,1,999,2,4,...,45.0,15,000411782470,1,28,#NE#,PMDB,01120,AC,PR
6,2004,13,NaN,3,18,-1,1,999,2,2,...,15120.0,15,001022542470,1,43,#NE#,PMDB,01120,AC,PR
7,2004,13,NaN,3,10,-1,1,601,2,3,...,13113.0,13,001750702445,1,23,#NE#,PT,01120,AC,ES
8,2004,13,NaN,3,10,-1,1,601,2,2,...,13613.0,13,001580082461,1,18,#NE#,PT,01120,AC,PR
9,2004,13,NaN,1,9,-1,1,999,2,4,...,43333.0,43,003552582461,1,8,#NE#,PV,01120,AC,RO


In [10]:
candidatos = (
    df_candidatos[[
        'NOME_CANDIDATO',
        'DATA_NASCIMENTO',
        'CPF_CANDIDATO',
        'NUM_TITULO_ELEITORAL_CANDIDATO',
        'NM_EMAIL',
        'NOME_MUNICIPIO_NASCIMENTO',
        'SIGLA_UF_NASCIMENTO',
        'CODIGO_NACIONALIDADE',
        'COD_GRAU_INSTRUCAO',
        'CODIGO_OCUPACAO',
    ]]
    .drop_duplicates('CPF_CANDIDATO', keep='last')
    # .sort_values(by=['NOME_CANDIDATO'], ascending=True, na_position='first')
    .rename(columns = {
        'NOME_CANDIDATO': 'nome',
        'DATA_NASCIMENTO': 'data_nascimento',
        'CPF_CANDIDATO': 'cpf',
        'NUM_TITULO_ELEITORAL_CANDIDATO': 'titulo_eleitoral',
        'NM_EMAIL': 'email',
        'NOME_MUNICIPIO_NASCIMENTO': 'cidade',
        'SIGLA_UF_NASCIMENTO': 'estado',
        'CODIGO_NACIONALIDADE': 'nacionalidade',
        'COD_GRAU_INSTRUCAO': 'grau_instrucao',
        'CODIGO_OCUPACAO': 'ocupacao'
    })
)

In [11]:
candidatos

,nome,data_nascimento,cpf,titulo_eleitoral,email,cidade,estado,nacionalidade,grau_instrucao,ocupacao
0,JOÃO DE ALMEIDA,26081957,18139124249,000356682496,NaN,TENENTE PORTELA,RS,1,4,601
1,PAULO CÉSAR FERREIRA DE ARAUJO,08041957,07857136204,000525772445,NaN,ACRELÂNDIA,AC,1,3,601
3,ERAIDES CAETANO DE SOUZA,26081957,40917860900,002817512496,NaN,ACRELÂNDIA,AC,1,0,601
12,DAMARIS DE OLIVEIRA,24091966,49488929200,000968162410,NaN,UBIRATÃ,PR,1,6,999
14,EDILENE SILVA DOS REIS,07041971,38950707268,020802772232,NaN,CAARAPO,MS,1,3,999
16,EDNA BERNARDINO E SILVA,29061958,34006451253,001738842445,NaN,GUARACI,PR,1,8,143
18,ERACIDES CAETANO DE SOUZA,14011950,17630207972,002817542437,NaN,TAIO,SC,1,6,999
19,ESTEVÃO DE SOUZA FERREIRA,26121978,61072621215,003157012461,NaN,RIO BRANCO,AC,1,6,999
20,FRANCISCO ANTONIO DA SILVA DIAS,15061964,21580430244,000764552488,NaN,BOCA DO ACRE,AM,1,7,143
23,IVAN DA SILVA FERREIRA,23041974,44378793253,002187552488,NaN,SENA MADUREIRA,AC,1,6,999


Observações
- Algumas 'cidades de nascimento' são inválidas
- Algumas datas de nascimento estão com tipo float
- Alguns estados estão com tipo float


In [15]:
cnx = mysql.connect(**config)
cursor = cnx.cursor(buffered=True)

for candidato in candidatos.itertuples():
    # Buscar id da cidade
    query = ("SELECT id FROM cidade WHERE "
             "nome = '" + candidato.cidade + "' AND "
             "estado_id = (SELECT id FROM estado WHERE sigla = '" + str(candidato.estado) + "')")
    cursor.execute(query)
    cidades = cursor.fetchall()
    cidade_id = cidades[0][0] if len(cidades) > 0 else None

    # Cadastrar candidato
    novo_candidato = ("INSERT INTO candidato "
                   "(nome, data_nascimento, cpf, titulo_eleitoral, email, cidade_id, ocupacao_id, nacionalidade_id, grau_instrucao_id) "
                   "VALUES (%(nome)s, %(data_nascimento)s, %(cpf)s, %(titulo_eleitoral)s, %(email)s, %(cidade_id)s, %(ocupacao)s, %(nacionalidade)s, %(grau_instrucao)s)")
    cursor.execute(novo_candidato, {
        'nome': candidato.nome,
        'data_nascimento': formatar_data(str(candidato.data_nascimento)),
        'cpf': candidato.cpf,
        'titulo_eleitoral': candidato.titulo_eleitoral,
        'email': candidato.email if email_valido(candidato.email) else None,
        'cidade_id': cidade_id,
        'ocupacao': candidato.ocupacao if (int(candidato.ocupacao) >= 0 & int(candidato.ocupacao)) <= 999 else 0,
        'nacionalidade': candidato.nacionalidade if (int(candidato.nacionalidade) >= 0 & int(candidato.nacionalidade) <= 4) else 0,
        'grau_instrucao': candidato.grau_instrucao if (int(candidato.grau_instrucao) >= 0 & int(candidato.grau_instrucao) <= 8) else 0
    })
    cnx.commit()

cursor.close()
cnx.close()

formatar_data: nan


TypeError: must be str, not float

In [27]:
cnx = mysql.connect(**config)
cursor = cnx.cursor(buffered=True)
candidato = candidatos.iloc[18]
query = ("SELECT id FROM cidade WHERE "
             "nome = '" + candidato.cidade + "' AND "
             "estado_id = (SELECT id FROM estado WHERE sigla = '" + str(candidato.estado) + "')")
print(query)
cursor.execute(query)
cidades = cursor.fetchall()
cidade_id = cidades[0][0] if len(cidades) > 0 else None
print(cidade_id)
cursor.close()
cnx.close()

SELECT id FROM cidade WHERE nome = 'OLINDA ASSAD LETRA' AND estado_id = (SELECT id FROM estado WHERE sigla = 'PR')
None


In [310]:
candidatos.iloc[18].cidade

'OLINDA ASSAD LETRA'

In [22]:
print(type(str(candidatos.iloc[0].email)))
print(str(candidatos.iloc[0].email)[0])
print(type(candidatos.iloc[0].email))

<class 'str'>
n
<class 'float'>
